In [2]:
#QUERY_URL = "//div[@class='separator']//a/@href"
# QUERY_URL = "//h2[@class='inner-item__title']//a/@href"

In [1]:
import random
import requests
from requests_html import HTMLSession
from requests_html import HTML
import os
import json
import validators
from time import sleep
from functools import partial


In [1]:
from os import mkdir, listdir
import pandas as pd
from time import sleep
from os.path import join, isfile, isdir, exists, abspath
from tqdm import tqdm, trange
import re
from functools import reduce
from PIL import Image


In [5]:
class Crawler:

    USER_AGENT_LIST = [
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1",
        "Mozilla/5.0 (X11; CrOS i686 2268.111.0) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.57 Safari/536.11",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1092.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1090.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/19.77.34.5 Safari/537.1",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.9 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.0) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.36 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_0) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.0 Safari/536.3",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24"
    ]

    def __init__(self):

        # ruta XPATH hacia los enlaces que queremos recopilar
        self.query_extract_all_href = QUERY_URL

        # variable que permite listar todos los enlaces recopilado
        self.all_json_to_send = []
        pass

    def new_session_get(self):
        session = HTMLSession()
        header = dict()
        header['user-agent'] = random.choice(self.USER_AGENT_LIST)
        return partial(session.get, headers=header)

    def start_request(self, url):
        session_get = self.new_session_get()
        response = session_get(url)
        return response

    def parse(self, response):
        self.all_json_to_send = []
        all_extracted_href = response.html.xpath(self.query_extract_all_href)
        for href in all_extracted_href:
            json_to_send = dict()
            #path = "https://www.hindustantimes.com"
            # json_to_send["url"]=path+href
            json_to_send["url"] = href
            self.all_json_to_send.append(json_to_send)


In [6]:
SEED_CRAWL = 'https://www.cnnchile.com/category/cultura/page/'
QUERY_URL = "//h2[@class='inner-item__title']//a/@href"
MAX_PAGE_CULTURA = 32
LAST_ID = 17798719
STARTING_ID_CULTURA = 18000000

crawler = Crawler()

In [8]:
url_noticias = []
for page in trange(MAX_PAGE_CULTURA, desc='url noticias'):
    response = crawler.start_request(SEED_CRAWL + str(page))
    crawler.parse(response)
    noticias = [obj['url']
                for obj in crawler.all_json_to_send if 'cultura' in obj['url']]
    url_noticias += noticias

len(url_noticias)


url noticias: 100%|██████████| 32/32 [00:29<00:00,  1.09it/s]


364

In [9]:
noticia_a = url_noticias[4]
response = crawler.start_request(noticia_a)
noticia_dict = {'url': noticia_a}
noticia_a


'https://www.cnnchile.com/cultura/muere-almudena-grandes-a-los-61-anos_20211127/'

In [10]:
QUERY_TITLE = "//h1[@class='main-single-header__title']/text()"
noticia_dict['title'] = response.html.xpath(QUERY_TITLE)[0]
response.html.xpath(QUERY_TITLE)[0]


'Muere la reconocida escritora española Almudena Grandes a los 61 años'

In [13]:
QUERY_BAJADA = "//div[@class='main-single-header__excerpt']//p/text()"
noticia_dict['text'] = re.sub(
    u'\xa0', u' ', response.html.xpath(QUERY_BAJADA)[0])
noticia_dict['text']

'La escritora murió a los 61 años a causa del cáncer que dio a conocer en una  columna publicada en el diario El País. El presidente del gobierno español, Pedro Sánchez y el artista Alejandro Sanz lamentaron su partida a través de redes sociales. '

In [14]:
QUERY_CONTENT = "//div[@class='main-single-body__content']//p"
new_lines = response.html.xpath(QUERY_CONTENT)[0].text.splitlines()
re.match(r'^(.*)(– )(.*)', new_lines[0]).groups()

('(CNN Español) ',
 '– ',
 'La reconocida escritora española Almudena Grandes falleció este sábado, así lo confirmó El País, diario en el cual Grandes se desempeñaba como columnista.')

In [15]:
lines = [line.text for line in response.html.xpath(QUERY_CONTENT)]
new_lines = [re.sub(u'\xa0', u' ', line)
             for line in lines if not line.startswith('Lee también:')]
new_lines[0] = re.match(r'^(.*– )?(.*)', new_lines[0]).groups()[1]
noticia_dict['text'] += ' '.join(new_lines)

In [16]:
noticia_dict

{'url': 'https://www.cnnchile.com/cultura/muere-almudena-grandes-a-los-61-anos_20211127/',
 'title': 'Muere la reconocida escritora española Almudena Grandes a los 61 años',
 'text': 'La escritora murió a los 61 años a causa del cáncer que dio a conocer en una  columna publicada en el diario El País. El presidente del gobierno español, Pedro Sánchez y el artista Alejandro Sanz lamentaron su partida a través de redes sociales. La reconocida escritora española Almudena Grandes falleció este sábado, así lo confirmó El País, diario en el cual Grandes se desempeñaba como columnista. Grandes, de 61 años, murió debido a un cáncer que ella mismo dio a conocer a través de una de sus columnas. La autora de “Las edades de Lulú” y “La madre de Frankestein” era una de las escritoras españolas más prolíferas de los últimos años. Fue ganadora de varios galardones, entre ellos el Premio Nacional de Narrativa en 2018 en España y el Premio Iberoamericano de Novela Elena Poniatowska. Su muerte causó la r

In [ ]:
noticias_cultura = []
_id = STARTING_ID_CULTURA
for i, url in enumerate(tqdm(url_noticias)):
    _id = STARTING_ID_CULTURA + i
    response = crawler.start_request(url)
    _dict = {'id': _id,
             'url':  url}
    _dict['title'] = response.html.xpath(QUERY_TITLE)[0]
    _dict['text'] = re.sub(u'\xa0', u' ', response.html.xpath(QUERY_BAJADA)[0])
    lines = [line.text for line in response.html.xpath(QUERY_CONTENT)]
    new_lines = [re.sub(u'\xa0', u' ', line)
                 for line in lines if not line.startswith('Lee también:')]
    new_lines[0] = re.match(r'^(.*– )?(.*)', new_lines[0]).groups()[1]
    _dict['text'] += ' '.join(new_lines)
    noticias_cultura.append(_dict)
    sleep(5)
noticias_cultura[:3]


In [115]:
df = pd.DataFrame(noticias_cultura)
df

,id,url,title,text
0,18000000,https://www.cnnchile.com/cultura/musico-joan-m...,Joan Manuel Serrat anunció su retiro,La despedida del artista comenzará en el Beaco...
1,18000001,https://www.cnnchile.com/cultura/rihanna-obtie...,Rihanna obtiene la designación de “heroína nac...,"La cantante fue homenajeada en su país natal, ..."
2,18000002,https://www.cnnchile.com/cultura/400-peliculas...,Desde “El Húsar de la Muerte” hasta la actuali...,La cineteca de la casa de estudios celebra 60 ...
3,18000003,https://www.cnnchile.com/cultura/cordillera-su...,"“La cordillera de los sueños”, de Patricio Guz...",La 36ª edición de los Premios Goya se realizar...
4,18000004,https://www.cnnchile.com/cultura/muere-almuden...,Muere la reconocida escritora española Almuden...,La escritora murió a los 61 años a causa del c...
...,...,...,...,...
359,18000359,https://www.cnnchile.com/cultura/calamaro-mon-...,“Dios los Cría”: Calamaro invita a Mon Laferte...,"La chilena interpretará ""Tantas veces"" junto a..."
360,18000360,https://www.cnnchile.com/cultura/paloma-mami-v...,“Qué Weá”: Paloma Mami se reivindica como chil...,La cantante participó en la dirección del vide...
361,18000361,https://www.cnnchile.com/cultura/ellen-degener...,Ellen DeGeneres informó que planea terminar su...,"La popular animadora explicó que ""cuando eres ..."
362,18000362,https://www.cnnchile.com/cultura/entrevista-do...,Los poemas en la UCI del Dr. Glenn Hernández: ...,Casi un año antes del primer muerto por la pan...


In [116]:
df.to_csv('extra_cultura.csv', index=False)

In [19]:
# trabajo manual, buscar tags que puedan estar relacionados con ecologia y planeta
tags_ecologia_planeta = [('animales', 8),            # total 19
                         ('maltrato-animal', 4),     # total 6
                         ('especies', 2),            # total 2
                         ('crisis-climatica', 4),    # total 4
                         ('cambio-climatico', 4),    # total 34
                         ('emergencia-climatica', 3),# total 3
                         ('vida-silvestre', 1),      # total 1
                         ('humedales', 1),            # total 1
                         ('fauna', 5),               # total 5
                         ('medioambiente', 5),       # total 8
                         ('medio-ambiente', 5),      # total 22
                         #('greta_thornberryXD', 7),
                         ]

In [20]:

QUERY_CONTENT = "//div[@class='main-single-body__content']//p"
QUERY_TITLE = "//h1[@class='main-single-header__title']/text()"
QUERY_BAJADA = "//div[@class='main-single-header__excerpt']//p/text()"

In [21]:
# obtener las url de las noticias relacionadas con la categoria
url_noticias = []
STARTING_ID_PLANETA = 19000000

for (tag_name, num_pages) in tqdm(tags_ecologia_planeta):
    SEED_CRAWL = f'https://www.cnnchile.com/tag/{tag_name}/page/'
    QUERY_URL = "//h2[@class='inner-item__title']//a/@href"
    crawler = Crawler()
    for page in range(1, num_pages+1):
        response = crawler.start_request(SEED_CRAWL + str(page))
        crawler.parse(response)
        noticias = [obj['url'] for obj in crawler.all_json_to_send]
        url_noticias += noticias

print(len(url_noticias))
url_noticias = list(set(url_noticias))  # eliminar posibles duplicados
len(url_noticias)


100%|██████████| 11/11 [00:36<00:00,  3.28s/it]

420


383

In [22]:
# obtener las noticias en si
noticias_planeta = []
for i, url in enumerate(tqdm(url_noticias)):
    # algunas noticias son de futuro360 y con esas se muere
    # no se si hay mas sitios distintos a cnn asi que su match noma
    if re.match(r'^.*//www.(\w+)\..*', url).groups()[0] == 'cnnchile':
        _id = STARTING_ID_PLANETA + i
        response = crawler.start_request(url)
        _dict = {'id': _id,
                'url':  url}
        _dict['title'] = response.html.xpath(QUERY_TITLE)[0]
        _dict['text'] = re.sub(u'\xa0', u' ', response.html.xpath(QUERY_BAJADA)[0])
        lines = [line.text for line in response.html.xpath(QUERY_CONTENT)]
        new_lines = [re.sub(u'\xa0', u' ', line)
                    for line in lines if not line.startswith('Lee también:')]
        new_lines[0] = re.match(r'^(.*– )?(.*)', new_lines[0]).groups()[1]
        _dict['text'] += ' '.join(new_lines)
        noticias_planeta.append(_dict)
        sleep(5) # porsiaca
noticias_planeta[:3]

 40%|████      | 155/383 [15:05<22:11,  5.84s/it]


IndexError: list index out of range

In [30]:
len(noticias_planeta)

381

In [31]:

df = pd.DataFrame(noticias_planeta)
df

,id,url,title,text
0,19000000,https://www.cnnchile.com/mundo/bolsonaro-maltr...,Ley contra el maltrato animal: Bolsonaro promu...,"Antes de firmar la normativa, el presidente de..."
1,19000001,https://www.cnnchile.com/pais/chile-ciudad-men...,Chile tiene la ciudad menos contaminada en Lat...,Así como tenemos 9 de las 10 ciudades más cont...
2,19000002,https://www.cnnchile.com/pais/rostros-marcha-c...,Los rostros de la marcha contra el cambio clim...,La sequía y el temor por el panorama que deber...
3,19000003,https://www.cnnchile.com/mundo/informe-ipcc-ca...,5 posibles escenarios climáticos que advierte ...,"Olas de calor marina, fuertes precipitaciones ..."
4,19000004,https://www.cnnchile.com/cop25/pinera-cop25-pa...,Pdte. Piñera tras COP25: “Los países que más c...,El mandatario fue enfático en señalar que “se ...
...,...,...,...,...
376,19000378,https://www.cnnchile.com/mundo/encontraron-un-...,Encontraron un pendrive en las fecas de una fo...,El hallazgo fue realizado por unos investigado...
377,19000379,https://www.cnnchile.com/pais/ministerio-medio...,Ministerio de Medio Ambiente lanzó estrategia ...,"La jefa de la cartera, Carolina Schmidt, desta..."
378,19000380,https://www.cnnchile.com/mundo/papa-francisco-...,Papa Francisco en el Día de la Tierra: “No ten...,El pontífice llamó a la humanidad a mantenerse...
379,19000381,https://www.cnnchile.com/lodijeronencnn/coment...,Raúl Sohr por informe de EE.UU. sobre el cambi...,"El informe, realizado por los servicios de int..."


In [32]:

# este contiene noticias de un rango mayor de fechas
# hacer un drop duplicates por title (y categoria) al momento de unir con el
# ds original
df.to_csv('extra_ecologia-planeta.csv', index=False)

In [5]:
import pandas as pd
df_paso3 = pd.read_csv('dataset_paso_3.csv')
print(len(df_paso3))
df_cultura = pd.read_csv('./extra_cultura.csv')
df_cultura['content'] = df_cultura[['title', 'text']].agg('. '.join, axis=1)
df_cultura['category'] = '6-Cultura_y_Artes'

df_planeta = pd.read_csv('./extra_ecologia-planeta.csv')
df_planeta['content'] = df_planeta[['title', 'text']].agg('. '.join, axis=1)
df_planeta['category'] = '8-Ecología_y_Planeta'

45514


In [6]:
df_cultura[['content', 'category']]

,content,category
0,Joan Manuel Serrat anunció su retiro. La despe...,6-Cultura_y_Artes
1,Rihanna obtiene la designación de “heroína nac...,6-Cultura_y_Artes
2,Desde “El Húsar de la Muerte” hasta la actuali...,6-Cultura_y_Artes
3,"“La cordillera de los sueños”, de Patricio Guz...",6-Cultura_y_Artes
4,Muere la reconocida escritora española Almuden...,6-Cultura_y_Artes
...,...,...
359,“Dios los Cría”: Calamaro invita a Mon Laferte...,6-Cultura_y_Artes
360,“Qué Weá”: Paloma Mami se reivindica como chil...,6-Cultura_y_Artes
361,Ellen DeGeneres informó que planea terminar su...,6-Cultura_y_Artes
362,Los poemas en la UCI del Dr. Glenn Hernández: ...,6-Cultura_y_Artes


In [7]:
df_final = pd.concat([df_paso3, df_cultura, df_planeta])
len(df_final)

46259

In [8]:
df_final.drop_duplicates(subset=['title'], keep='last', inplace=True)
len(df_final)

46101

In [9]:
df_final.groupby(['category']).size().sort_values()

category
6-Cultura_y_Artes              1133
8-Ecología_y_Planeta           1183
5-Catástrofes_y_Accidentes     1243
2-Economía                     2635
10-Salud                       2683
4-Ciencias_y_Tecnología        4773
7-Deporte                      6702
9-Crimen_delitos_y_Justicia    8212
1-Mundo                        8586
3-Política_y_Conflictos        8951
dtype: int64

In [10]:
df_final.to_csv('dataset_paso3d5.csv', index=False)

In [ ]:
freno() # 3.5 v2

In [11]:
df_paso3 = pd.read_csv('dataset_paso_3.csv')
print(len(df_paso3))
df_cultura = pd.read_csv('./extra_cultura.csv')
df_cultura['content'] = df_cultura[['title', 'text']].agg('. '.join, axis=1)
df_cultura['category'] = '6-Cultura_y_Artes'
df_final = pd.concat([df_paso3, df_cultura])
df_final.drop_duplicates(subset=['title'], keep='last', inplace=True)
df_final.to_csv('dataset_paso3d8.csv', index=False)
len(df_final)

45514


45723

In [12]:
df_paso3 = pd.read_csv('dataset_paso_3.csv')
print(len(df_paso3))
df_planeta = pd.read_csv('./extra_ecologia-planeta.csv')
df_planeta['content'] = df_planeta[['title', 'text']].agg('. '.join, axis=1)
df_planeta['category'] = '8-Ecología_y_Planeta'
df_final = pd.concat([df_paso3, df_planeta])
df_final.drop_duplicates(subset=['title'], keep='last', inplace=True)
df_final.to_csv('dataset_paso3d9.csv', index=False)
len(df_final)

45514


45748